In [ ]:
# TODO: Derive cleaning events by looking at speed over grond for january and july as well as the following efficiency indicators:
#       - Propeller Slip (most direct indicator)
#       - Speed Efficiency Ratio (df['speed_per_power'] = df['Hull Over Ground Speed'] / df['Propeller Shaft Mechanical Power'])
#       - Fuel Consumption vs Speed (df['fuel_per_speed'] = df['Consumption for Propulsion'] / df['Hull Over Ground Speed'])
#       - Power-to-Speed Relationship (Vessel Propeller Shaft Mechanical Power relative to Vessel Hull Over Ground Speed)

